In the course of building my Teensy-based rollercoasterometer, I hooked two GPS receivers up to it:

* A uBlox NEO-M8U
* An Adafriut FeatherWing Ultimate GPS

Over the night of 26/27 Jun 2020, these both were recorded for about 10 hours while I went to bed.

What can we make of the statistics of these recordings?

First, we need to read the NMEA and extract the coordinates. This is in the GxRMC sentences in the record. However, the first thing to note is that some of the sentences were corrupted in transit. Also, there was at least one bad point in the NEO-M8U record. So, we will check the checksum, then reject all 

In [ ]:
import numpy as np

def calc_checksum(sentence):
    checksum=0
    
    for i in range(1,len(sentence)-2):
        if sentence[i]=="*":
            break
        checksum^=ord(sentence[i])
    return "%02x"%checksum
        
def is_valid(sentence):
    try:
        starpos=sentence.index("*")
    except:
        return False #No checksum marker
    calc=calc_checksum(sentence)
    store=sentence[starpos+1:starpos+3]
    return calc==store

Now we need to be able to convert a value in the NMEA encoded DDDMM.MMMMMMM to decimal degrees

In [ ]:
def dddmm_to_deg(dddmm):
    ddd=np.floor(dddmm/100)
    mm=dddmm-(ddd*100)
    return ddd+mm/60

Finally, read a file:

In [ ]:
def read_gprmc(infn):
    with open(infn) as inf:
        lat=[]
        lon=[]
        alt=[]
        for line in inf:
            if is_valid(line):
                if line[1]=="G" and line[3:6]=="GGA":
                    try:
                        parts=line.split(",")
                        #for i,part in enumerate(parts):
                        #    print(i,part)
                        this_lat=dddmm_to_deg(float(parts[2]))
                        this_lon=-dddmm_to_deg(float(parts[4]))
                        this_alt=float(parts[9])
                    except:
                        continue
                    if this_lat>40 and this_lon<-105:
                        lat.append(this_lat)
                        lon.append(this_lon)
                        alt.append(this_alt)
    return (lat,lon,alt)


In [ ]:
(lat,lon,alt)=read_gprmc("/mnt/big/home/chrisj/workspace/Data/teensy/LOG00103/NMEA.nmea")

In [ ]:
print(len(lat),len(lon),len(alt))

In [ ]:
import matplotlib.pyplot as plt
%notebook matplotlib
plt.plot(lon,lat,'.')